정의된 Relation의 개수 비교

In [31]:
import json
from ratsql.utils import registry
from collections import defaultdict


def read_json(file):
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    return json_data

def read_jsonl(file):
    with open(file) as json_file:
        json_data = [json.loads(line) for line in json_file]
    
    return json_data

In [38]:
spider_data_loc = 'rat-sql/data/spider/train_spider.json'

train_spider = read_json(spider_data_loc)

In [40]:
len(train_spider)

7000

In [ ]:
spider_data_loc = 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink/enc/train.jsonl'

spider_train = read_jsonl(spider_data_loc)

In [29]:
spider_data_loc = 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink/enc/train.jsonl'

spider_train = read_jsonl(spider_data_loc)

In [43]:
spider_train[0].keys()

dict_keys(['raw_question', 'question', 'db_id', 'sc_link', 'cv_link', 'columns', 'tables', 'table_bounds', 'column_to_table', 'table_to_columns', 'foreign_keys', 'foreign_keys_tables', 'primary_keys'])

In [44]:
a = {'1':0, '2': 3}

In [51]:
spider_train[0]

{'raw_question': 'How many heads of the departments are older than 56 ?',
 'question': ['how',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?'],
 'db_id': 'department_management',
 'sc_link': {'q_col_match': {'2,7': 'CPM',
   '2,12': 'CPM',
   '5,1': 'CPM',
   '5,11': 'CPM'},
  'q_tab_match': {'2,1': 'TEM', '5,0': 'TEM'}},
 'cv_link': {'num_date_match': {'9,1': 'NUMBER',
   '9,4': 'NUMBER',
   '9,5': 'NUMBER',
   '9,6': 'NUMBER',
   '9,7': 'NUMBER',
   '9,10': 'NUMBER',
   '9,11': 'NUMBER',
   '9,12': 'NUMBER'},
  'cell_match': {},
  'value_match': {},
  'value_word': {}},
 'columns': [['*', '<type: text>'],
  ['department', 'id', '<type: number>'],
  ['name', '<type: text>'],
  ['creation', '<type: text>'],
  ['ranking', '<type: number>'],
  ['budget', 'in', 'billions', '<type: number>'],
  ['nu', '##m', 'employees', '<type: number>'],
  ['head', 'id', '<type: number>'],
  ['name', '<type: text>'],
  ['born', 'state', '<type: text>'],

In [53]:
target_relation = {}
target_relations = ['sc_link', 'cv_link', 'q_col_match', 'q_tab_match', 'num_date_match', 'cell_match', 'foreign_keys', 'primary_keys', ]

target_relations = ['NUMBER', 'TIME', 'CELL_MATCH', 'CPM', 'CEM', 'TPM', 'TEM', 'PRIMARY', 'FOREIGN']
for t in target_relations:
    target_relation[t] = 0
for example in spider_train:
    sc_link = example['sc_link']
    cv_link = example['cv_link']

    foreign_keys = len(example['foreign_keys'])
    foreign_keys_tables = example['foreign_keys_tables']
    primary_keys = len(example['primary_keys'])

    q_col_match = sc_link['q_col_match']
    q_tab_match = sc_link['q_tab_match']

    num_date_match = cv_link['num_date_match']
    cell_match = cv_link['cell_match']

    number = len([i for i in num_date_match.values() if i == 'NUMBER'])
    time = len([j for j in num_date_match.values() if j == 'TIME'])
    cell_match = len([j for j in cell_match.values() if j == 'CELLMATCH'])

    CPM = len([i for i in q_col_match.values() if i == 'CPM'])
    CEM = len([i for i in q_col_match.values() if i == 'CEM'])
    
    TPM = len([i for i in q_tab_match.values() if i == 'TPM'])
    TEM = len([i for i in q_tab_match.values() if i == 'TEM'])

    target_relation['CPM'] += CPM
    target_relation['CEM'] += CEM
    target_relation['TPM'] += TPM
    target_relation['TEM'] += TEM
    target_relation['NUMBER'] += number
    target_relation['TIME'] += time
    target_relation['CELL_MATCH'] += cell_match
    target_relation['PRIMARY'] += primary_keys
    target_relation['FOREIGN'] += foreign_keys

target_relation

{'NUMBER': 32758,
 'TIME': 3101,
 'CELL_MATCH': 7248,
 'CPM': 38754,
 'CEM': 18136,
 'TPM': 4248,
 'TEM': 11678,
 'PRIMARY': 49412,
 'FOREIGN': 52246}

In [24]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from pyvis.network import Network
from sklearn.metrics.pairwise import cosine_similarity

import itertools
from collections import defaultdict

import scipy
import sys
sys.path.append('rat-sql')
sys.path.append('../')

import attr
import _jsonnet
import json 

from ratsql.commands.infer import Inferer
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "3"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"

os.chdir('/repo')

@attr.s
class InferConfig:
    config = attr.ib()
    config_args = attr.ib()
    logdir = attr.ib()
    section = attr.ib()
    beam_size = attr.ib()
    output = attr.ib()
    step = attr.ib()
    use_heuristic = attr.ib(default=False)
    mode = attr.ib(default="infer")
    limit = attr.ib(default=None)
    output_history = attr.ib(default=False)

bs="6"
qv_link="false"
dist_relation="true"
orthog="false"
orth_init="false"
bi_match="true"
bi_way="true"

infer_config = InferConfig(config='rat-sql/configs/spider/nl2code-bert.jsonnet', 
                            config_args='{"att": 1, "bert_lr": 3e-06, "bert_token_type": true, "bert_version": "bert-large-uncased-whole-word-masking", "bs": 6, "clause_order": null, "cv_link": true, "data_path": "rat-sql/data/spider/", "decoder_hidden_size": 512, "end_lr": 0, "end_with_from": true, "loss": "label_smooth", "lr": 0.000744, "max_steps": 121000, "num_batch_accumulated": 4, "num_layers": 8, "qv_link": '+qv_link+ ', "dist_relation": ' + dist_relation + ', "use_orthogonal": ' + orthog + ', "use_orth_init": '+orth_init+', "sc_link": true, "summarize_header": "avg", "use_align_loss": true, "use_align_mat": true, "use_column_type": false, "bi_match": '+bi_match+', "bi_way": '+bi_way+'}', 
                            logdir='rat-sql/logdir/bert_run', 
                            section='val',
                            beam_size=1, 
                            output='__LOGDIR__/ie_dirs/bert_run_true_1-step1.infer', 
                            step=1, use_heuristic=True, mode='infer', limit=None, output_history=False)

config = json.loads(_jsonnet.evaluate_file(infer_config.config, tla_codes={'args': infer_config.config_args}))

inferer = Inferer(config)

logdir = os.path.join(infer_config.logdir, config['model_name'])
step = 60500

model = inferer.load_model(logdir, step)

relation_k_emb_0 = model.state_dict()['encoder.encs_update.encoder.layers.0.relation_k_emb.weight'].cpu()
relation_k_emb_1 = model.state_dict()['encoder.encs_update.encoder.layers.1.relation_k_emb.weight'].cpu()
relation_k_emb_2 = model.state_dict()['encoder.encs_update.encoder.layers.2.relation_k_emb.weight'].cpu()
relation_k_emb_3 = model.state_dict()['encoder.encs_update.encoder.layers.3.relation_k_emb.weight'].cpu()
relation_k_emb_4 = model.state_dict()['encoder.encs_update.encoder.layers.4.relation_k_emb.weight'].cpu()
relation_k_emb_5 = model.state_dict()['encoder.encs_update.encoder.layers.5.relation_k_emb.weight'].cpu()
relation_k_emb_6 = model.state_dict()['encoder.encs_update.encoder.layers.6.relation_k_emb.weight'].cpu()
relation_k_emb_7 = model.state_dict()['encoder.encs_update.encoder.layers.7.relation_k_emb.weight'].cpu()

relation_ks = [relation_k_emb_0, relation_k_emb_1, relation_k_emb_2, relation_k_emb_3, relation_k_emb_4, relation_k_emb_5, relation_k_emb_6, relation_k_emb_7]

relation_v_emb_0 = model.state_dict()['encoder.encs_update.encoder.layers.0.relation_v_emb.weight'].cpu()
relation_v_emb_1 = model.state_dict()['encoder.encs_update.encoder.layers.1.relation_v_emb.weight'].cpu()
relation_v_emb_2 = model.state_dict()['encoder.encs_update.encoder.layers.2.relation_v_emb.weight'].cpu()
relation_v_emb_3 = model.state_dict()['encoder.encs_update.encoder.layers.3.relation_v_emb.weight'].cpu()
relation_v_emb_4 = model.state_dict()['encoder.encs_update.encoder.layers.4.relation_v_emb.weight'].cpu()
relation_v_emb_5 = model.state_dict()['encoder.encs_update.encoder.layers.5.relation_v_emb.weight'].cpu()
relation_v_emb_6 = model.state_dict()['encoder.encs_update.encoder.layers.6.relation_v_emb.weight'].cpu()
relation_v_emb_7 = model.state_dict()['encoder.encs_update.encoder.layers.7.relation_v_emb.weight'].cpu()

relation_vs = [relation_v_emb_0, relation_v_emb_1, relation_v_emb_2, relation_v_emb_3, relation_v_emb_4, relation_v_emb_5, relation_v_emb_6, relation_v_emb_7]

relation_index = {('qq_dist', -2): 0,
                    ('qq_dist', -1): 1,
                    ('qq_dist', 0): 2,
                    ('qq_dist', 1): 3,
                    ('qq_dist', 2): 4,
                    'qc_default': 5,
                    'qt_default': 6,
                    'cq_default': 7,
                    'cc_default': 8,
                    'cc_foreign_key_forward': 9,
                    'cc_foreign_key_backward': 10,
                    'cc_table_match': 11,
                    ('cc_dist', -2): 12,
                    ('cc_dist', -1): 13,
                    ('cc_dist', 0): 14,
                    ('cc_dist', 1): 15,
                    ('cc_dist', 2): 16,
                    'ct_default': 17,
                    'ct_foreign_key': 18,
                    'ct_primary_key': 19,
                    'ct_table_match': 20,
                    'ct_any_table': 21,
                    'tq_default': 22,
                    'tc_default': 23,
                    'tc_primary_key': 24,
                    'tc_table_match': 25,
                    'tc_any_table': 26,
                    'tc_foreign_key': 27,
                    'tt_default': 28,
                    'tt_foreign_key_forward': 29,
                    'tt_foreign_key_backward': 30,
                    'tt_foreign_key_both': 31,
                    ('tt_dist', -2): 32,
                    ('tt_dist', -1): 33,
                    ('tt_dist', 0): 34,
                    ('tt_dist', 1): 35,
                    ('tt_dist', 2): 36,
                    'qcCEM': 37,
                    'cqCEM': 38,
                    'qtTEM': 39,
                    'tqTEM': 40,
                    'qcCPM': 41,
                    'cqCPM': 42,
                    'qtTPM': 43,
                    'tqTPM': 44,
                    'qcNUMBER': 45,
                    'cqNUMBER': 46,
                    'qcTIME': 47,
                    'cqTIME': 48,
                    'qcCELLMATCH': 49,
                    'cqCELLMATCH': 50}

relation_index_list = list(relation_index.keys())

print(f'bs: {bs}, qv_link: {qv_link}, dist_relation: {dist_relation}, orthog: {orthog}, orth_init: {orth_init}, bi_match: {bi_match}, bi_way: {bi_way}')

WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bert_version': 'bert-large-uncased-whole-word-masking', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'rat-sql/data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'rat-sql/data/spider/nl2code,output_from=true,fs=2,emb=bert,cvlink'}}
WARNING <class 'ratsql.models.spider.spider_enc.SpiderEncoderBert'>: superfluous {'enc_qv_link': False, 'qv_link': False, 'use_orthogonal': False}


Used Relations: 51


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from rat-sql/logdir/bert_run/bs=6,qv_link=false,dist=true,orthog=false,orth_init=false,bi_way=true,bi_match=true/model_checkpoint-00060500


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 39.59 GiB total capacity; 9.01 GiB already allocated; 11.19 MiB free; 9.02 GiB reserved in total by PyTorch)

In [13]:
model_preproc = registry.instantiate(
                registry.lookup('model', config['model']).Preproc,
                config['model'],
                unused_keys=('name',))

In [14]:
model_preproc

In [15]:
train_data = model_preproc.dataset('train')

In [19]:
desc = train_data[0]

In [21]:
desc[0].get('cv_link')

{'num_date_match': {'9,1': 'NUMBER',
  '9,4': 'NUMBER',
  '9,5': 'NUMBER',
  '9,6': 'NUMBER',
  '9,7': 'NUMBER',
  '9,10': 'NUMBER',
  '9,11': 'NUMBER',
  '9,12': 'NUMBER'},
 'cell_match': {},
 'value_match': {},
 'value_word': {}}

In [11]:
train_eval_data_loader = torch.utils.data.DataLoader(
            train_data,
            batch_size=self.train_config.eval_batch_size,
            collate_fn=lambda x: x)

{('qq_dist', -2): 0,
 ('qq_dist', -1): 1,
 ('qq_dist', 0): 2,
 ('qq_dist', 1): 3,
 ('qq_dist', 2): 4,
 'qc_default': 5,
 'qt_default': 6,
 'cq_default': 7,
 'cc_default': 8,
 'cc_foreign_key_forward': 9,
 'cc_foreign_key_backward': 10,
 'cc_table_match': 11,
 ('cc_dist', -2): 12,
 ('cc_dist', -1): 13,
 ('cc_dist', 0): 14,
 ('cc_dist', 1): 15,
 ('cc_dist', 2): 16,
 'ct_default': 17,
 'ct_foreign_key': 18,
 'ct_primary_key': 19,
 'ct_table_match': 20,
 'ct_any_table': 21,
 'tq_default': 22,
 'tc_default': 23,
 'tc_primary_key': 24,
 'tc_table_match': 25,
 'tc_any_table': 26,
 'tc_foreign_key': 27,
 'tt_default': 28,
 'tt_foreign_key_forward': 29,
 'tt_foreign_key_backward': 30,
 'tt_foreign_key_both': 31,
 ('tt_dist', -2): 32,
 ('tt_dist', -1): 33,
 ('tt_dist', 0): 34,
 ('tt_dist', 1): 35,
 ('tt_dist', 2): 36,
 'qcCEM': 37,
 'cqCEM': 38,
 'qtTEM': 39,
 'tqTEM': 40,
 'qcCPM': 41,
 'cqCPM': 42,
 'qtTPM': 43,
 'tqTPM': 44,
 'qcNUMBER': 45,
 'cqNUMBER': 46,
 'qcTIME': 47,
 'cqTIME': 48,
 'qc

In [ ]:
model.encoder.encs_update.compute_relations